In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi, My name is Seonghwan, I live in South Korea, Seoul",
            "Cool! How's the weather in Seoul?")

In [10]:
add_message("Seoul is now hot, sweaty sunny day",
            "Be careful to heat, stay inside")

In [11]:
get_history()

{'history': 'Seonghwan from Seoul, South Korea, describes the weather as hot and sweaty. The AI advises to be careful of the heat and stay inside to stay cool.'}